In [1]:
import pandas as pd
import numpy as np
import pyarrow.feather as feather


In [2]:
channel_df=pd.read_csv('/mnt/g/ADA/df_channels_en.tsv',sep='\t')

In [9]:
channel_df.head()

,category_cc,join_date,channel,name_cc,subscribers_cc,videos_cc,subscriber_rank_sb,weights
0,Gaming,2010-04-29,UC-lHJZR3Gqxm24_Vd_AJ5Yw,PewDiePie,101000000,3956,3.0,2.087
1,Education,2006-09-01,UCbCmjCuTUZos6Inko4u57UQ,Cocomelon - Nursery ...,60100000,458,7.0,2.087
2,Entertainment,2006-09-20,UCpEhnqL0y41EpW2TvWAHD7Q,SET India,56018869,32661,8.0,2.087
3,Howto & Style,2016-11-15,UC295-Dw_tDNtZXFeAPAW6Aw,5-Minute Crafts,60600000,3591,9.0,2.087
4,Sports,2007-05-11,UCJ5v_MCY6GNUBTO8-D3XoAg,WWE,48400000,43421,11.0,2.087


In [3]:
channel_df['category_cc'].value_counts()

category_cc
Music                    24285
Entertainment            22951
Gaming                   20143
People & Blogs           18413
Howto & Style            11875
Education                 7803
Film and Animation        6875
Sports                    5148
Science & Technology      4864
Comedy                    3767
Autos & Vehicles          3705
News & Politics           2263
Travel & Events           1989
Pets & Animals            1292
Nonprofits & Activism      969
Name: count, dtype: int64

In [4]:
# sample 10% samples from each category_cc
channel_df_sampled=channel_df.groupby('category_cc').apply(lambda x: x.sample(frac=0.1)).reset_index(drop=True)


/tmp/ipykernel_22513/317488887.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  channel_df_sampled=channel_df.groupby('category_cc').apply(lambda x: x.sample(frac=0.1)).reset_index(drop=True)


In [6]:
channel_df_sampled.value_counts('category_cc')

category_cc
Music                    2428
Entertainment            2295
Gaming                   2014
People & Blogs           1841
Howto & Style            1188
Education                 780
Film and Animation        688
Sports                    515
Science & Technology      486
Comedy                    377
Autos & Vehicles          370
News & Politics           226
Travel & Events           199
Pets & Animals            129
Nonprofits & Activism      97
Name: count, dtype: int64

In [7]:
# save the sampled dataframe
feather.write_feather(channel_df_sampled,'data/df_channels_en_sampled.feather')

In [10]:
# save the sampled channel ids
channel_df_sampled['channel'].to_csv('data/channel_ids_sampled.csv',index=False)

In [11]:
del channel_df,channel_df_sampled

In [2]:
channel_id=pd.read_csv('data/channel_ids_sampled.csv')

In [14]:
timeseries_df=pd.read_csv('/mnt/g/ADA/df_timeseries_en.tsv',sep='\t')

In [17]:
timeseries_df.count()

channel         18872499
category        18851915
datetime        18872499
views           18872499
delta_views     18872499
subs            18872499
delta_subs      18872499
videos          18872499
delta_videos    18872499
activity        18872499
dtype: int64

In [15]:
sample_timeseries_df=timeseries_df[timeseries_df['channel'].isin(channel_id['channel'])]

In [16]:
sample_timeseries_df.count()

channel         1880664
category        1880664
datetime        1880664
views           1880664
delta_views     1880664
subs            1880664
delta_subs      1880664
videos          1880664
delta_videos    1880664
activity        1880664
dtype: int64

In [18]:
sample_timeseries_df.to_feather('data/df_timeseries_en_sampled.feather')

In [19]:
del timeseries_df,sample_timeseries_df

In [4]:
channel_id_lst=channel_id['channel'].tolist()

In [3]:
metadata=feather.read_table('/mnt/g/ADA/yt_metadata_helper.feather')

In [5]:
metadata.column_names

['categories',
 'channel_id',
 'dislike_count',
 'display_id',
 'duration',
 'like_count',
 'upload_date',
 'view_count']

In [6]:
import pyarrow.parquet as pq

# Define a function to filter and convert to pandas DataFrame
def filter_metadata(metadata, channel_id_lst):
    # Convert pyarrow Table to pandas DataFrame
    metadata_df = metadata.to_pandas()
    # Filter the DataFrame
    filtered_df = metadata_df[metadata_df['channel_id'].isin(channel_id_lst)]
    return filtered_df

# Process metadata in chunks
chunk_size = 10000  # Define the chunk size
filtered_metadata_df_list = []

for i in range(0, metadata.num_rows, chunk_size):
    chunk = metadata.slice(i, chunk_size)
    filtered_chunk = filter_metadata(chunk, channel_id_lst)
    filtered_metadata_df_list.append(filtered_chunk)

# Concatenate all filtered chunks into a single DataFrame
filtered_metadata_df = pd.concat(filtered_metadata_df_list, ignore_index=True)


In [11]:
filtered_metadata_df.to_feather('data/yt_metadata_en_sampled.feather')

In [13]:
del metadata,filtered_metadata_df,filtered_metadata_df_list,channel_id_lst,channel_id

In [2]:
num_comments_df=pd.read_csv('/mnt/g/ADA/num_comments.tsv',sep='\t')

In [2]:
metadata_df=pd.read_feather('data/yt_metadata_en_sampled.feather')

In [3]:
display_id_lst=metadata_df['display_id'].tolist()

In [4]:
del metadata_df

In [6]:
filtered_num_comments_df=num_comments_df[num_comments_df['display_id'].isin(display_id_lst)]

In [7]:
del num_comments_df

In [8]:
filtered_num_comments_df.to_feather('data/num_comments_sampled.feather')